In [ ]:
import os
cwd=os.getcwd().replace('\\','/')
#github/willzjc.github.io
if 'historical' not in cwd:
    cwd=cwd+'/github/willzjc.github.io/python/historical_price/'
os.chdir(cwd)

In [2]:
import os
import pandas as pd
import numpy as np
import datetime

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF
import cufflinks as cf

plotly.tools.set_credentials_file(username='willzjc@gmail.com', api_key='nMap3MiO6fJx7AO5PnzT')

import sys

import matplotlib.pyplot as plt

from scipy.interpolate import spline
from matplotlib import dates as mdates
import matplotlib
# matplotlib.use('Qt5Agg')

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

def drawdf(df,draw_plot=True,product='unknown'):


    #Normalize DF First

    df['combined']=0

    for eachitem in df.columns:
        if not 'date' in eachitem:
            df[eachitem] = (df[eachitem] - df[eachitem].mean()) / (df[eachitem].std())
            df['combined']=df['combined']+df[eachitem]

    # Get Total Rating based on average of
    df['average_sell_price']=df[[c for c in df.columns[1:]]].mean(axis=1)

    df.set_index(pd.DatetimeIndex(df['date'])).resample('3M').mean().plot()


    # plot with all members
    # df.set_index(pd.DatetimeIndex(df['date'])).resample('M').plot()

    # df=df.replace('\sUSD','',regex=True).apply(pd.to_numeric, errors='ignore')
    # df['dateindex']=pd.to_datetime(df['date'])
    df['date']=pd.to_datetime(df['date'])
    df=df.set_index(pd.DatetimeIndex(df['date']))



    cpi=pd.read_csv('ref/AUCPI',delimiter='\t')

    print cpi

    cpi['date']=pd.to_datetime(cpi['date'])

    cpi['CPI'] = (cpi['CPI'] - cpi['CPI'].mean()) / (cpi['CPI'].std())

    # Timeshift
    timeshift=30
    print cpi
    print df
    templatecpi=cpi.copy()
    templatedf = df.copy()
    # for timeshift in [0,30,60,90,120]:
    corrs={}
    corrslist=[]
    rng=range(-150,150)
    if draw_plot:
        rng=[-91,0,91]
        
    dfs=[]

    for timeshift in rng:
        cpi=templatecpi.copy()
        df=templatedf.copy()


        cpi['date'] = cpi['date'] - datetime.timedelta(days=(timeshift))
        cpi=cpi.set_index(pd.DatetimeIndex(cpi['date']))

        # Normalize CPI
        # index = pd.date_range(cpi['date'].max(),cpi['date'].min())
        # index = pd.date_range(cpi['date'])
        # values = pd.Series(cpi.values, index=index)

        # Read above link about the different Offset Aliases, S=Seconds
        # resampled_values = values.resample('2.5D')
        # cpi.diff()  # compute the difference between each point!
        df.plot(y='average_sell_price',title="timeshift: " + str(timeshift))
        ax = cpi.plot(y='CPI')
        df.resample('M').mean().plot(y='average_sell_price',ax=ax,title="timeshift: " + str(timeshift))


        # xnew = np.linspace(df['combined'].min(), df['combined'].max(), 300)  # 300 represents number of points to make between T.min and T.max

        # power_smooth = spline(df['date'], df['combined'], xnew)

        # plt.plot(xnew, power_smooth)

        #

        ##### Concat both tables #####
        combined_df=pd.concat([cpi,df],axis=1, join='inner')
        # print type(df),type(cpi)

        # ##### Massaging Data #####
        # ### resample
        # df = df.resample('M').mean()
        # df = df.resample('M')
        # df = df.resample('D')
        # # ### interpolate
        # df= df.interpolate(method='cubic')

        # df.plot(kind='bar')
        # df['bad_rate'].plot(secondary_y=True)

        # #### PLOTTING SECTION #####
        # df = df.resample('M').mean()
        # df = df.resample('D')
        # tsint = df.interpolate(method='cubic')
        # cpi=cpi.interpolate(method='cubic')
        # tsint.plot()
        # tsint.plot(y=['combined','average_sell_price'], use_index=True)
        # tsint.plot()
        # ax = cpi.plot(y='CPI', use_index=True,color='r',legend=True)
        # tsint.plot.line(ax=ax,y='average_sell_price',use_index=True,secondary_y=True,legend=True)
        # cpi_s.plot(ax=ax)
        # result.plot(y='average_sell_price',kind='bar')
        # result.plot(y='CPI',secondary_y=True)
        corr=combined_df.CPI.corr(combined_df.average_sell_price)
        corrs[timeshift]=corr
        corrslist.append(corr)
        
        dfs.append(combined_df)

        if draw_plot:
            f, axarr = plt.subplots(2)

            combined_df['positive']=combined_df['average_sell_price'] > 0

            datefmt = mdates.DateFormatter('%Y-%m-%d')

            axarr[0].format_xdata = datefmt
            axarr[1].format_xdata = datefmt

            axarr[0].xaxis.set_major_formatter(datefmt)
            axarr[1].xaxis.set_major_formatter(datefmt)

            combined_df.CPI.plot(ax=axarr[1], legend=True)
            combined_df.average_sell_price.plot(legend=True, ax=axarr[0],kind='bar'
             ,color=combined_df.positive.map({True: 'g', False: 'r'}),title='Timeshift: '+ str(timeshift) + ' days')


            # combined_df.iplot(kind='scatter', filename='cufflinks-cf-simple-line')

            # py.iplot([{
            #     'x': combined_df.index,
            #     'y': combined_df[col],
            #     'name': col
            # } for col in combined_df.columns if 'date' not in col ], filename='cufflinks-simple-line')

    cdf=None
    correlation_pickle='correl.pickle'
    if os.path.exists(correlation_pickle):
        cdf=pd.read_pickle(correlation_pickle)
        # if not product in cdf.columns:
        # cdf[product]=corrslist

    else:
        cdf=pd.DataFrame(columns=['offset',product])
        cdf['offset']=range(-150,150)
        cdf[product] = corrslist
        # for k in corrs.keys():
        #     cdf[k] = corrs[k]

    cdf.to_pickle(correlation_pickle)
    cdf['total']=0
    for c in cdf.columns:
        if 'offset' not in c:
            cdf['total']=cdf['total']+cdf[c]



    cdf.to_csv('correl.csv')

    # print cdf

    # if draw_plot:
    #     plt.gcf().autofmt_xdate()
    #     plt.show()

def readfiles():

    bol_recursive=False
    df=None

    basepath=os.getcwd().replace('\\','/') + '/' + 'ref' + '/'
    print basepath

    files=[]

    for str_dirname, lst_subdirs, lst_files in os.walk(basepath):
        if not bol_recursive:
            while len(lst_subdirs) > 0:
                lst_subdirs.pop()
                for file in lst_files:
                    if '.csv' in file and not 'corr' in file:
                        with open(basepath+file,'rb') as f:
                            files.append(f.read())
                            f.close()

    for f in files:

        headers = {}
        buffer=[]
        headermode = True
        for line in f.split('\n'):
            if headermode and 'Date,' in line:
                headermode=False
            elements=line.strip().split(',')
            if len(elements) < 3 and len(elements) > 1:         # filter out header info
                headers[elements[0].strip()]=elements[1].strip()
            else:
                linein=False
                if len(elements)>1 and not headermode and '0.00 USD,0.00 USD' not in line:
                    buffer.append(line)
                    linein=True
                # print f,linein,line

        # Read file stream CSV
        currentdf = pd.read_csv(StringIO('\n'.join(buffer)))

        # Replace Strings
        currentdf = currentdf.replace('\sUSD', '', regex=True).apply(pd.to_numeric, errors='ignore')
        try:
            if df==None:
                df=pd.DataFrame(columns=['date'])
                df['date'] = currentdf['Date']
        except:
            # TODO
            print 'Deal with this exception another time'

        # print currentdf.columns
        df[headers['Keywords']] = currentdf['Average Selling Price']

    df['combined']=0
    df.to_csv('fileread.csv')
    return df

def main():
    df=readfiles()
    init_df=df.copy()
    drawdf(df,draw_plot=True,product='alcohol')

    init_df['datetime'] = pd.to_datetime(init_df['date'])
    init_df=init_df.set_index(pd.DatetimeIndex(init_df['date']))

    # init_df.set_index('date')

    print init_df.resample('3M').mean


if __name__ == "__main__":
    main()

C:/Users/Will-AMD/PycharmProjects/github/willzjc.github.io/python/historical_price/ref/
Deal with this exception another time
Deal with this exception another time
Deal with this exception another time
Deal with this exception another time
Deal with this exception another time


Deal with this exception another time
Deal with this exception another time
Deal with this exception another time
Deal with this exception another time


           date    CPI
0   01-Mar-2015  117.3
1   01-Jun-2015  118.7
2   01-Sep-2015  120.2
3   01-Dec-2015  123.4
4   01-Mar-2016  124.5
5   01-Jun-2016  125.7
6   01-Sep-2016  127.1
7   01-Dec-2016  130.7
8   01-Mar-2017  132.1
9   01-Jun-2017  133.1
10  01-Sep-2017  136.0
11  01-Dec-2017  140.3
         date       CPI
0  2015-03-01 -1.415562
1  2015-06-01 -1.219830
2  2015-09-01 -1.010117
3  2015-12-01 -0.562729
4  2016-03-01 -0.408940
5  2016-06-01 -0.241170
6  2016-09-01 -0.045438
7  2016-12-01  0.457873
8  2017-03-01  0.653605
9  2017-06-01  0.793414
10 2017-09-01  1.198858
11 2017-12-01  1.800035
                 date      beer     scotch     vodka   whiskey      wine  \
date                                                                       
2015-04-15 2015-04-15 -0.496580  -0.143757 -0.403335 -0.335554 -0.924887   
2015-04-16 2015-04-16 -0.560201  -0.021203 -0.495216 -0.354368  0.510849   
2015-04-17 2015-04-17 -0.454165  -0.017911 -0.062464 -0.352035  0.774299   
2015-04-2

     offset   clothes      food  furniture     total
0      -150 -0.457397  0.194598   0.275466  0.025334
1      -149 -0.052805  0.002868  -0.574141 -1.248157
2      -148  0.010161  0.489916  -0.889631 -0.779107
3      -147  0.244229  0.053040  -0.655714 -0.716890
4      -146  0.453572 -0.399850  -0.334434 -0.561422
5      -145  0.063708 -0.667226  -0.131057 -1.469151
6      -144  0.199373 -0.035601  -0.158151  0.011242
7      -143 -0.202886  0.059609   0.099210 -0.088135
8      -142  0.178157 -0.257261   0.073821 -0.010567
9      -141  0.229880 -0.135140   0.149621  0.488721
10     -140 -0.096075 -0.736257   0.036017 -1.592631
11     -139  0.130628 -0.614745  -0.339656 -1.647546
12     -138 -0.015738 -0.313356   0.772569  0.886950
13     -137 -0.286233 -0.732508  -0.521243 -3.079969
14     -136 -0.525419 -0.155073  -0.991253 -3.343491
15     -135  0.285465  0.016706   0.135542  0.875427
16     -134  0.084840  0.360096  -0.031511  0.826851
17     -133  0.269334 -0.310558   0.431265  0.